In [6]:
#Doc pour pedalboard
#https://colab.research.google.com/drive/1bHjhJj1aCoOlXKl_lOfG99Xs3qWVrhch#scrollTo=ISJ6VFc4yz9-
#https://github.com/spotify/pedalboard


In [7]:
import soundfile as sf
import IPython.display as ipd
import random
from os import listdir
from os.path import isfile, join
from pedalboard import (
    Pedalboard,
    Convolution,
    Compressor,
    Chorus,
    Distortion,
    Gain,
    Reverb,
    Limiter,
    LadderFilter,
    Phaser,
    load_plugin,
)

import yaml
import tqdm

In [8]:
#Exemple d'utilisation de pedalboard

#LOAD VST
# Load a VST3 package from a known path on disk:
vst = load_plugin("VST/Carve v2.5.0/Windows/Carve.vst3")

print(vst.parameters.keys())

vst.pregain1 = 1
vst.pregain2 = 1
vst.postgain1 = 1
vst.postgain2 = 1

# Use this VST to process some audio:
audio, sample_rate = sf.read('data_subset/keyboard_acoustic_004-083-127.wav')

"""
effected = vst(audio, sample_rate=sample_rate)
"""

# ...or put this VST into a chain with other plugins:
board = Pedalboard([vst], sample_rate=sample_rate)
# ...and run that pedalboard with the same VST instance!
effected = board(audio)

# Run the audio through this pedalboard!
effected = board(audio)

# Write the audio back as a wav file:
with sf.SoundFile('./processed-output-stereo.wav', 'w', samplerate=sample_rate, channels=len(effected.shape)) as f:
    f.write(effected)

ipd.Audio(effected, rate=sample_rate)

dict_keys(['mode1', 'pregain1', 'postgain1', 'tweak1', 'mode2', 'pregain2', 'postgain2', 'tweak2', 'routing', 'stereo', 'drylevel', 'outputgain', 'bypass'])


In [9]:
#EXAMPLE POUR RANDOMIZER LES PARAMETRES DU VST
#vst.parameters.keys()

#########################
#RANDOMIZE VST PARAMETERS
#########################
for parameter_name in vst.parameters:
    #print(parameter_name)
    #print(vst.parameters[parameter_name])

    #Get min and max values for the parameter

    exec("min_value=vst.%s.min_value" % (parameter_name))
    exec("max_value=vst.%s.max_value" % (parameter_name))
    #print("min_value=", min_value)
    #print("max_value=", max_value)

    #print(type(min_value))
    #Si l'argument est un nombre, on lui donne une valeur aléatoire
    if type(min_value) is float or type(min_value) is int:
        #Note : value gets mapped automatically to the closest step
        random_value= random.uniform(min_value, max_value)   
        exec("vst.%s=random_value" % (parameter_name))
        exec("actual_value=vst.%s" % (parameter_name))
        
    #Si l'argument est un  Bool, génère un bool aléatoire    
    if type(min_value) is bool:
        random_value=bool(random.getrandbits(1))
        exec("vst.%s=random_value" % (parameter_name))
        exec("actual_value=vst.%s" % (parameter_name))
        
    print(parameter_name,"=", actual_value)

mode1 = 1.0
pregain1 = 0.0
postgain1 = 0.5
tweak1 = 0.2
mode2 = 4.0
pregain2 = 1.0
postgain2 = 0.4
tweak2 = 1.0
routing = 0.5
stereo = False
drylevel = 0.6
outputgain = 0.7
bypass = False


## Distorteur de données

On lui donne en entrée le répertoire qui contient les fichiers .wav à traiter, on crée une liste de VSTs qu'on aimerait générer.

Le programme applique à chaque fichier, pour chaque VST, la distortion, et stock les fichiers distordus. 

In [10]:
### THE REAL CODE ###

data_directory='First subset'
storage_directory= 'First distorted dataset'#For output files
nb_files_to_generate_for_each_input_files = 1


#LOAD VSTs
# Load a VST3 package from a known path on disk:
liste_vst=[]

vst1 = load_plugin("VST/Carve v2.5.0/Windows/Carve.vst3")
liste_vst.append(vst1)

##################################################
#Loop to generate files with random VST parameters
##################################################

#Get all files names in the directory:
all_files = [f[0:(len(f)-4)] for f in listdir(data_directory)]


#Pour chaque Plugin de distorsion 
for num_vst, current_vst in enumerate(liste_vst):
    print("VST ",num_vst)
    # Create pedalboard containing only the VST
    board = Pedalboard([current_vst], sample_rate=sample_rate)


    # Randomize all the vst parameters 
    for i in range(nb_files_to_generate_for_each_input_files) :
        param_values = {}
        for parameter_name in current_vst.parameters:
            #print(parameter_name)
            #print(vst.parameters[parameter_name])

            #Get min and max values for the parameter
            exec("min_value=current_vst.%s.min_value" % (parameter_name))
            exec("max_value=current_vst.%s.max_value" % (parameter_name))
            #print("min_value=", min_value)
            #print("max_value=", max_value)

            #print(type(min_value))
            #Si l'argument est un nombre, on lui donne une valeur aléatoire
            if type(min_value) is float or type(min_value) is int:
                #Note : value gets mapped automatically to the closest step
                param_value= random.uniform(min_value, max_value)   
                exec("current_vst.%s=param_value" % (parameter_name))
                exec("actual_value=current_vst.%s" % (parameter_name))

            #Si l'argument est un Bool, génère un bool aléatoire    
            if type(min_value) is bool:
                param_value=bool(random.getrandbits(1))
                exec("current_vst.%s=param_value" % (parameter_name))
                exec("actual_value=current_vst.%s" % (parameter_name))

            #Si l'argument est Bypass, on ne veut pas désactiver l'effet...
            if parameter_name == 'bypass' :
                param_value = False
                exec("current_vst.%s=param_value" % (parameter_name))
                exec("actual_value=current_vst.%s" % (parameter_name))
            
            #Si l'argument est drylevel, on veut uniquement l'effet (et pas le son dry)
            if parameter_name == 'drylevel' :
                param_value = 0.
                exec("current_vst.%s=param_value" % (parameter_name))
                exec("actual_value=current_vst.%s" % (parameter_name))

            #print(parameter_name,"=", actual_value)
            param_values[parameter_name] = param_value
            
            # Save the disto parameters in a dictionnary
            with open('param_values.yml', 'w') as outfile:
                yaml.dump(param_values, outfile, default_flow_style=False)

        #Apply randomized preset to the all file (generates X output files)
        for filename in tqdm.tqdm(all_files) :
            #LOAD FILE
            audio, sample_rate = sf.read(data_directory+"/"+filename+".wav")
            #print(filename)

            ########################################
            # Run the audio through this pedalboard!
            effected = board(audio)

            #####################################
            # Write the audio back as a wav file:
            storage_name=filename+"_VST_"+str(num_vst)+"_"+str(i)+".wav"
            with sf.SoundFile("./"+storage_directory+"/"+storage_name, 'w', samplerate=sample_rate, channels=len(effected.shape)) as f:
                f.write(effected)
                
print("DONE")

VST  0


100%|████████████████████████████████████████████████████████████████████████████| 12678/12678 [02:42<00:00, 77.87it/s]

DONE
